In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [4]:
import mlflow

In [9]:
from sklearn.model_selection import train_test_split

In [18]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [26]:
from urllib.parse import urlparse

In [7]:
data = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv",sep=';')

In [11]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [12]:
train, test = train_test_split(data)

In [15]:
target = 'quality'

In [16]:
train_x = train.drop([target], axis=1)
test_x = test.drop([target], axis=1)
train_y = train[[target]]
test_y = test[[target]]

In [20]:
def eval_metrics(real, pred):
    rmse = np.sqrt(mean_squared_error(real, pred))
    mae = mean_absolute_error(real, pred)
    r2 = r2_score(real, pred)
    return rmse, mae, r2

In [32]:
with mlflow.start_run() as run:
    lr = LinearRegression()
    lr.fit(train_x, train_y)
    
    predict = lr.predict(test_x)
    
    (rmse, mae, r2) = eval_metrics(test_y, predict)
    
    mlflow.log_metric("rmse",rmse)
    mlflow.log_metric("r2",r2)
    mlflow.log_metric("mae",mae)
    
    tracking_url = urlparse(mlflow.get_tracking_uri()).scheme
    
    if tracking_url != 'file':
        mlflow.sklearn.log_model(lr, 'model', registered_model_name='LinearRegressionModel')
    else:
        mlflow.sklearn.log_model(lr, 'model')

In [33]:
mlflow.get_run(run_id=run.info.run_id)

<Run: data=<RunData: metrics={'mae': 0.5181263704570884, 'r2': 0.3617973445671461, 'rmse': 0.662919572621345}, params={}, tags={'mlflow.log-model.history': '[{"run_id": "ebafe65a0e3b4a7fa002b51b067c62e0", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2023-06-17 16:48:11.011035", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.9.12", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.2.2", '
                             '"serialization_format": "cloudpickle", "code": '
                             'null}}, "model_uuid": '
             